# Analyzing risk of a stock market portfolio

We will implement the Value at Risk (VaR) and Conditional Value at Risk (CVaR) using:<br>
- Historical method
- Parametric method
- Monte Carlo method

In [53]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr

Now, we write the function to import the data. Our data includes the name of the stock, start date and end date

In [54]:
def get_data(stocks, start, end):
    stock_data = pdr.get_data_yahoo(stocks, start=start, end=end)
    stock_data = stock_data['Close']
    returns = stock_data.pct_change()
    mean_return = returns.mean()
    cov_matrix = returns.cov()
    return returns, mean_return, cov_matrix

From the data from the function above, we calculate the performance of the portfolio

In [55]:
def portfolio_performance(weights, mean_return, cov_matrix, time):
    returns = np.sum(mean_return * weights) * time
    sd = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(time)
    return returns, sd

Now we instantiate our portfolio

In [56]:
stocklist = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock + '.AX' for stock in stocklist]
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=800)

In [57]:
returns, mean_return, cov_matrix = get_data(stocks, start_date, end_date)
returns = returns.dropna()
print(returns)

Symbols       CBA.AX    BHP.AX    TLS.AX    NAB.AX    WBC.AX    STO.AX
Date                                                                  
2019-04-04 -0.004773 -0.005073 -0.003049 -0.014303 -0.006862 -0.010386
2019-04-08 -0.003103  0.017083  0.000000 -0.004434 -0.006526  0.022489
2019-04-09 -0.005943  0.003510  0.009174 -0.004453 -0.001546  0.029325
2019-04-10  0.007117 -0.003497 -0.003030  0.001627  0.002322 -0.004274
2019-04-11 -0.006926 -0.006267  0.006079 -0.007714 -0.012741  0.004292
...              ...       ...       ...       ...       ...       ...
2021-06-07 -0.005755  0.002051 -0.002793 -0.031625 -0.008932 -0.002591
2021-06-08 -0.004415 -0.009621 -0.002801  0.004880  0.003755  0.005195
2021-06-09 -0.003942  0.005994  0.002809 -0.007845 -0.006734 -0.009044
2021-06-10  0.007618 -0.007602  0.000000  0.002259  0.001507 -0.009126
2021-06-11 -0.004811  0.013458  0.002801 -0.006011 -0.011282  0.009211

[554 rows x 6 columns]


We randomize the weights of the stocks in our portfolio by using np.random

In [58]:
weights = np.random.random(len(returns.columns))
weights /= np.sum(weights)

Now, we associate the weights for the stocks in our portfolio

In [59]:
returns['portfolio'] = returns.dot(weights)
print(returns)

Symbols       CBA.AX    BHP.AX    TLS.AX    NAB.AX    WBC.AX    STO.AX  \
Date                                                                     
2019-04-04 -0.004773 -0.005073 -0.003049 -0.014303 -0.006862 -0.010386   
2019-04-08 -0.003103  0.017083  0.000000 -0.004434 -0.006526  0.022489   
2019-04-09 -0.005943  0.003510  0.009174 -0.004453 -0.001546  0.029325   
2019-04-10  0.007117 -0.003497 -0.003030  0.001627  0.002322 -0.004274   
2019-04-11 -0.006926 -0.006267  0.006079 -0.007714 -0.012741  0.004292   
...              ...       ...       ...       ...       ...       ...   
2021-06-07 -0.005755  0.002051 -0.002793 -0.031625 -0.008932 -0.002591   
2021-06-08 -0.004415 -0.009621 -0.002801  0.004880  0.003755  0.005195   
2021-06-09 -0.003942  0.005994  0.002809 -0.007845 -0.006734 -0.009044   
2021-06-10  0.007618 -0.007602  0.000000  0.002259  0.001507 -0.009126   
2021-06-11 -0.004811  0.013458  0.002801 -0.006011 -0.011282  0.009211   

Symbols     portfolio  
Date         

### Historical VaR

The formula for VaR is: <br><br> 
$$VaR_a(X) = -inf\{x \in \mathbb{R} | F_x (x) > a\} = F_{Y}^{-1} (1 - a)$$

![Image](https://upload.wikimedia.org/wikipedia/commons/6/64/VaR_diagram.JPG)

Some theory:<br>
- If $X$ is a random variable for the Profit and Loss distribution, $Y = -X(loss\ distribution)$
- $F_{x} (x)$ is the CDF of the Profit and Loss distribution, $F_{Y}^{-1} (y)$ is the inverse CDF of loss distribution

Now we write a function to calculate the historical VaR. We want to read the returns dataframe and output the percentile of the distribution at the given alpha confidence interval

In [60]:
def historical_var(returns, alpha=5):
    if isinstance(returns, pd.Series):
        return np.percentile(returns, alpha)
    elif isintance(returns, pd.DataFrame):
        return returns.aggregate(historical_var, alpha=5)
    else:
        raise TypeError

Function to calculate historical cVaR (aka expected short fall):

![Image](https://i.ytimg.com/vi/qML5RpE7mxw/maxresdefault.jpg)

In [61]:
def historical_cvar(returns, alpha=5):
    if isinstance(returns, pd.Series):
        below_var = returns <= historical_var(returns, alpha)
        return returns[below_var].mean()
    elif isinstance(returns, pd.DataFrame):
        return returns.aggregate(historical_cvar, alpha=5)
    else:
        raise TypeError

In [62]:
time = 1

var = -historical_var(returns['portfolio'], 5) * np.sqrt(time)
cvar = -historical_cvar(returns['portfolio'], 5) * np.sqrt(time)
returns, sd = portfolio_performance(weights, mean_return, cov_matrix, time)

initial_investment = 100000
print('Expected portfolio return: ', round(initial_investment * returns, 2))
print('Value at risk at 95% confidence interval: ', round(initial_investment * var, 2))
print('Conditional value at risk at 95% confidence interval: ', round(initial_investment * cvar, 2))

Expected portfolio return:  42.36
Value at risk at 95% confidence interval:  2413.26
Conditional value at risk at 95% confidence interval:  4665.0
